### Обучите нейронную сеть любой архитектуры которой не было на курсе, либо обучите нейронную сеть разобранной архитектуры, но на том датасете, которого не было на уроках. Сделайте анализ, того, что вам помогло в улучшения работы нейронной сети.

Я использовал нашу модель для генерации текстов с минимальными изменениями (я использовал LSTM) и взял в качестве датасета небольшой отрывок из Шекспира (отсюда http://karpathy.github.io/2015/05/21/rnn-effectiveness/).<br>
Параметров не так много, но я в итоге пришёл к тому, что сильно увеличил размер скрытого слоя в LSTM ячейке.

In [1]:
import numpy as np
from tensorflow.keras.layers import Dense, Activation, LSTM
from tensorflow.keras.models import Sequential

In [2]:
with open("shakespear.txt", 'r', encoding='utf-8') as f:
    raw = f.read()
vocab = list(set(raw))
vocab_size = len(vocab)
print(vocab_size)

62


In [3]:
w2i = {}
for i, w in enumerate(vocab):
    w2i[w] = i
text_inds = np.array(list(map(lambda x: w2i[x], raw)))

In [4]:
SEQLEN = 10
STEP = 1

input_chars, label_chars = [], []
for i in range(0, len(raw) - SEQLEN, STEP):
    input_chars.append(raw[i: i + SEQLEN])
    label_chars.append(raw[i + SEQLEN])

In [5]:
X = np.zeros((len(input_chars), SEQLEN, vocab_size), dtype=np.bool)
y = np.zeros((len(input_chars), vocab_size), dtype=np.bool)
for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, w2i[ch]] = 1
    y[i, w2i[label_chars[i]]] = 1

In [8]:
BATCH_SIZE = 32
HIDDEN_SIZE = 512
NUM_ITERATIONS = 25
NUM_EPOCHS_PER_ITERATION = 5
NUM_PREDS_PER_EPOCH = 100

In [9]:
model = Sequential()
model.add(
    LSTM(
        HIDDEN_SIZE,
        return_sequences=False,
        input_shape=(SEQLEN, vocab_size),
        unroll=True
    )
)
model.add(Dense(vocab_size))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

In [ ]:
# выполнение серий тренировочных и демонстрационных итераций 
for iteration in range(NUM_ITERATIONS):

    # для каждой итерации запуск передачи данных в модель 
    print("=" * 50)
    print("Итерация #: %d" % (iteration))
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)

    # Select a random example input sequence.
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]

    # для числа шагов предсказаний использование текущей тренируемой модели 
    # конструирование one-hot encoding для тестирования input и добавление предсказания.
    print("Генерация из посева: %s" % (test_chars))
    print(test_chars, end="")
    for i in range(NUM_PREDS_PER_EPOCH):

        # здесь one-hot encoding.
        X_test = np.zeros((1, SEQLEN, vocab_size))
        for j, ch in enumerate(test_chars):
            X_test[0, j, w2i[ch]] = 1

        # осуществление предсказания с помощью текущей модели.
        pred = model.predict(X_test, verbose=0)[0]
        y_pred = vocab[np.argmax(pred)]

        # вывод предсказания добавленного к тестовому примеру 
        print(y_pred, end="")

        # инкрементация тестового примера содержащего предсказание
        test_chars = test_chars[1:] + y_pred
print()

На гитхабе нельзя свернуть результат ячейки, поэтому я вынес в отдельный файл train_output.txt ход обучения, а тут продублирую предсказание на последней эпохе последней итерации.

In [26]:
test_chars = "m my husba"
for i in range(NUM_PREDS_PER_EPOCH):

    # здесь one-hot encoding.
    X_test = np.zeros((1, SEQLEN, vocab_size))
    for j, ch in enumerate(test_chars):
        X_test[0, j, w2i[ch]] = 1

    # осуществление предсказания с помощью текущей модели.
    pred = model.predict(X_test, verbose=0)[0]
    y_pred = vocab[np.argmax(pred)]

    # вывод предсказания добавленного к тестовому примеру 
    print(y_pred, end="")

    # инкрементация тестового примера содержащего предсказание
    test_chars = test_chars[1:] + y_pred

nd.

MARK ANTONY:
Thou dost digest you! If accident doth seen a seat
:s not pray their patience.'

K

А теперь я взял первые 10 символов из файла, который я использовал для обучения, и на их основании предсказал 1000 символов.

In [25]:
test_chars = "That, poor"
for i in range(1000):

    # здесь one-hot encoding.
    X_test = np.zeros((1, SEQLEN, vocab_size))
    for j, ch in enumerate(test_chars):
        X_test[0, j, w2i[ch]] = 1

    # осуществление предсказания с помощью текущей модели.
    pred = model.predict(X_test, verbose=0)[0]
    y_pred = vocab[np.argmax(pred)]

    # вывод предсказания добавленного к тестовому примеру 
    print(y_pred, end="")

    # инкрементация тестового примера содержащего предсказание
    test_chars = test_chars[1:] + y_pred

 contempt,
Branches: to thy respect have not said here I came Butt
With such an anointed two wisdoms, the day
Must stay and be so hot all ad is death forbid, and thought
The bright buds of mine own.

BIONDELLO:
Marry, there was savage.

DUKE OF YORK:
I'll poison friendship. I hope I was a truft or dost
have made him,
Good Thomas.

ALBANY:
A plague of self-sweet queen as the king
May let her, wished with my words,
And he that keeps some suitors in the cases of the way.

SICINIUS:
We will.

SALARINO:
O, move me, how haps would not lurk to the bell!
'Nobles. Out, three!' Omnon. Hath he not
 ended her huge envy.

VENTIDIUS:
Nor he, my lord.

TRINCULO:
My lord, it were two bodies, to discharge itself in't with my words,
And he that keeps some suitors in the cases of the way.

SICINIUS:
We will.

SALARINO:
O, move me, how haps would not lurk to the bell!
'Nobles. Out, three!' Omnon. Hath he not
 ended her huge envy.

VENTIDIUS:
Nor he, my lord.

TRINCULO:
My lord, it were two bodies, to disc

В целом не очень осмысленно и текст является перемешением словосочетаний из оригинального текста, но всё равно прикольно, что он запомнил фразы, и вдвойне прикольно, что он даже структуру шекспировских сонетов сохраняет!

### Сделайте краткий обзор какой-нибудь научной работы посвященной тому или иному алгоритму нейронных сетей, который не рассматривался на курсе. Проведите анализ: Чем отличается выбранная вами на рассмотрение архитектура нейронной сети от других архитектур? В чем плюсы и минусы данной архитектуры? Какие могут возникнуть трудности при применении данной архитектуры на практике? 

Я стал разбирать статью о BERT (https://arxiv.org/abs/1810.04805v2)

BERT (Bidirectional Encoder Representations from Transformers) - это модель, построенная на трансформерах, и предобучена на задачах, связанных с учёто всего контекста (и слева, и справа).<br><br>
Сразу небольшое отступление от статьи, что бы разобраться в терминологии.<br>
Чтобы можно было использовать машинное обучение в задачах, связанных с текстом, мы должны каким-то образом перевести текст и последовательность чисел. Самое простое - кажому слову из текста сопоставить конкретное число - индекс, тогда из текста полуич словарь связей конкретных слов и их индексов и массив чисел, который представляет из себя первоначальный текст, где все слова заменены в соответсвии с этим словарём. С таким уже можно работать, но мы знаем, что, к примеру, слова "стул" и "стулья" - это одно существительно в разных формах. Нельзя ли как-то указать компьютеру на эту особенности языка? Решение этой задачи в том, чтобы сопоставлять каждому слову не одно число, а целый вектор, да таким образом, чтобы у похожих слов вектора были близки друг к другу. Эта задача называется эмбедингом. Следующий шаг - представить целое предложение, состоящее из многих слов, в виде матрицы, которая будет учитывать отношения между словами в предложении. Эту задачу решают т.н. Энкодеры в составе моделей глубокого обучения. Есть разные архитектуры энкодеров, на основе RNN, CNN, а в трансформере от Open AI (на нем основан BERT) используется т.н. self-attention механизм. Их энкодер состоит из 2х слоёв и в первом слое (который и есть self-attention) вычисляется отношение текущего кодируемого слова ко всем остальным в предложении. Считается, что этот слой позволяет оценить похожесть слов между собой. Такая модель может быть расширена до т.н. multi-head attention - когда параллельно используется много слоёв self-attention, каждый из которых обучается воспринимать слова на основании разных аспектов. Про декодер в трансформере говорить не будем - он нам и не нужен. Если сделать хороший энкодер, обучить его на большом датасете, то затем его можно будет использовать в разных задачах и создавать и обучать придётся только часть декодера.<br><br>
Теперь вернёмся к статье. BERT - это модель, состоящая из нескольких энкодеров из трансформера (размерность выхода из энкодера равна размерности входа, поэтому их можно стакать много-много раз), которую предобучили на большом датасете для 2х задач NLP (об этом ниже) и выложили в общий доступ. Авторы презентуют 2 модели, BERTbase и BERTlarge, первая состоит из 6ти блоков-энкодеров, вторая из 12ти, некоторые другие параметры так же у этих моделей различаются, что в итоге приводит к 110 миллионам параметров у BERTbase и около 340 миллионов параметров у BERTlarge. Отмечу, что по числу параметров BERTbase сравним с OpenAI GPT, с которым авторы сравнивают свои результаты.<br>
Обучение проводилось для двух задач NLP. Однонаправленные модели используют половину контекста для предсказания слова. К примеру, используют первые N слов предложения для предсказания N+1. Создатели BERT изменили эту задачу, чтобы сеть использовала весь контекст. В тексте случайным образом заменялись слова на маску и модель училась предсказывать, какие слова должны быть на месте масок. Таким образом, модель имела в распоряжении весь текст сразу, т.е. контекст учитывался истинно полностью (что отличает BERT от другой модели ELMo, которая по сути состоит из 2х частей, одна предсказывает слово по контексту слева, другая по контексту справа, а затем результат конкатенируется).<br>
Вторая задача - предсказание связи текущего предложения со следующим. Для этого был набран набор пар предложений, половина этих пар состояла из предложений, которые следовали подряд в тексте, а вторая половина, где второе предложение было выбрано случайно из текста. Обучение для такой задачи позволяет использовать BERT для задач по типу генерации ответов на вопросы.
Чтобы использовать BERT, достаточно добавить выходной слой, который будет декодировать выход BERT в тот вид, который нужен для конкретной задачи, и обучать только этот слой. В итоге авторы получили результаты для большого числа стандартных задач NLP, которые сильно опережали предыдущих лидеров.<br><br>
Добавлю от себя интересный факт - вскоре после выхода BERT состоялся релиз второй версии GPT (OpenAI GPT-2) с рекордным числом параметров 1.5 миллиардов (т.е. где-то в 4.5 раз больше, чем у BERTlarge). По сути эта модель является генератором текста и она оказалось настолько крута, что авторы не стали выкладывать её в свободный доступ опасаясь того, что она будет использоваться для генерации фейковых новостей.